# 1.7 Consultas Multitabla (Subconsultas y CTEs)

**Resumen Ejecutivo**: Subconsultas y CTEs (Common Table Expressions) permiten lógica modular y legible. Aquí aprenderás a comparar métricas individuales con promedios globales y detectar clientes/productos activos.

---

## 🎯 Objetivos de Aprendizaje
- Usar subconsultas en SELECT, FROM y WHERE para cálculos contextuales.
- Simplificar consultas complejas con CTEs para reutilizar lógica.
- Diferenciar cuándo usar subconsultas vs JOINs o CTEs.

## 🧰 Prerrequisitos
- Haber completado `06_agregaciones_kpis.ipynb` y ejecutado `dataset_setup.sql`.
- Conocer agregaciones (SUM, AVG) y JOINs básicos.
- Comprender el concepto de subconsulta (query dentro de query).

---

## 📚 Conceptos Clave
- **Subconsulta escalar**: retorna un solo valor (usado en comparaciones).
- **Subconsulta correlacionada**: referencia columnas de la consulta externa (más lenta).
- **CTE (WITH)**: define consulta temporal reutilizable y mejora legibilidad.
- **EXISTS**: verifica existencia sin traer datos (más eficiente que IN para grandes volúmenes).

> ⚠️ Las subconsultas correlacionadas pueden ser costosas; evaluar reemplazo con JOIN o CTE cuando sea posible.

---

## 🧪 Ejemplos en Código

In [ ]:
-- Margen del producto vs margen promedio global (subconsulta en SELECT)
SELECT p.producto_id, p.nombre, (p.precio_unitario - p.costo_unitario) AS margen,
       (SELECT AVG(precio_unitario - costo_unitario) FROM dbo.dim_productos) AS margen_promedio
FROM dbo.dim_productos p;

In [ ]:
-- Clientes con unidades vendidas > promedio general (subconsulta correlacionada)
SELECT c.cliente_id, c.nombre,
       (SELECT SUM(v.cantidad) FROM dbo.fact_ventas v WHERE v.cliente_id = c.cliente_id) AS unidades_cliente
FROM dbo.dim_clientes c
WHERE (SELECT SUM(v2.cantidad) FROM dbo.fact_ventas v2 WHERE v2.cliente_id = c.cliente_id) >
      (SELECT AVG(sum_cantidad) FROM (SELECT cliente_id, SUM(cantidad) AS sum_cantidad FROM dbo.fact_ventas GROUP BY cliente_id) x);

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE (SELECT SUM(v.cantidad) FROM dbo.fact_ventas v WHERE v.producto_id = p.producto_id) >
      (SELECT AVG(sum_ing) FROM (SELECT producto_id, SUM(cantidad) AS sum_ing FROM dbo.fact_ventas GROUP BY producto_id) y);

In [ ]:
SELECT COUNT(*) AS clientes_activos
FROM dbo.dim_clientes c
WHERE EXISTS (SELECT 1 FROM dbo.fact_ventas v WHERE v.cliente_id = c.cliente_id);

In [ ]:
WITH ventas_por_cliente AS (
    SELECT cliente_id, SUM(cantidad) AS unidades
    FROM dbo.fact_ventas
    GROUP BY cliente_id
), promedio AS (SELECT AVG(unidades) AS unidades_promedio FROM ventas_por_cliente)
SELECT c.cliente_id, c.nombre, v.unidades
FROM dbo.dim_clientes c
JOIN ventas_por_cliente v ON c.cliente_id = v.cliente_id
CROSS JOIN promedio p
WHERE v.unidades > p.unidades_promedio;

---

## ⚠️ Errores Comunes
- Subconsultas redundantes que ejecutan la misma lógica múltiples veces (usar CTE).
- No considerar impacto de subconsultas correlacionadas en performance.
- Mezclar subconsultas en SELECT con agregaciones sin clarificar granularidad.


---

## 🧪 Ejercicios Guiados
- 🟢 Subconsulta en WHERE: productos con ingreso total > ingreso promedio por producto.
- 🟠 Conteo con EXISTS: número de clientes activos (con ventas).
- 🔴 Reescribir consulta de clientes > promedio usando CTE para mejorar legibilidad.

---

## ✅ Conclusiones
- Subconsultas permiten cálculos contextuales (comparar individuo vs promedio).
- CTEs mejoran legibilidad y performance al reutilizar lógica intermedia.
- EXISTS es más eficiente que IN para verificar existencia en grandes tablas.

---

## 🚀 Aplicación Práctica
- Identificar clientes top vs promedio para campañas premium.
- Detectar productos sin actividad reciente (subconsultas o LEFT JOIN).
- Construir reportes jerárquicos con CTEs recursivas (avanzado).

---

## 💼 Perspectiva de Negocio
- Segmentar clientes/productos por desempeño relativo (arriba/debajo del promedio).
- Automatizar alertas de clientes inactivos para equipos de retención.
- Modularizar consultas complejas reduce errores y facilita auditorías.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 1  
**Módulo**: 1.7 Consultas Multitabla (Subconsultas y CTEs)  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](06_agregaciones_kpis.ipynb) | [Siguiente ➡️](08_calidad_datos_basica.ipynb)
---
